In [ ]:
# used gymnasium instead of gym, couldnt get the version deconflict to work with newer version of numpy

import gymnasium as gym 
from gymnasium import logger as gymlogger
from gymnasium.wrappers import RecordVideo  

import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt

import math
import glob
import io
import base64
from IPython.display import HTML
from IPython import display as ipythondisplay


In [121]:
def show_video():
    mp4list = glob.glob('video/*.mp4')
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io.open(mp4,'r+b').read()
        ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
    else: 
        print("Could not find video")

In [122]:
env = gym.make("CartPole-v1")

Task 1: Developing a RL agent

In [123]:
def rand_policy_agent(observation):
    return random.randint(0, 1)

In [ ]:
observation, info = env.reset()

action = rand_policy_agent(observation)


observation, reward, terminated, truncated, info = env.step(rand_policy_agent(1))


done = terminated or truncated

print("Observation:", observation)
print("Chosen action:", action)


Observation: [ 0.02913812  0.19420277 -0.04840008 -0.3038564 ]
Chosen action: 1


Task 2: Demonstrating effectivenes of RL agent
